##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Quantização de números inteiros pós-treinamento

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/lite/performance/post_training_integer_quant"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/lite/performance/post_training_integer_quant.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/lite/performance/post_training_integer_quant.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fonte no GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/lite/performance/post_training_integer_quant.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
</table>

## Visão geral

A quantização de números inteiros é uma estratégia de otimização que converte números de ponto flutuante de 32 bits (como pesos e saídas de ativação) para os números de ponto fixo de 8 bits mais próximos. Isso resulta em um modelo menor e maior velocidade de inferência, que é importante para dispositivos de baixa energia como [microcontroladores](https://www.tensorflow.org/lite/microcontrollers). Esse formato de dados também é exigido por aceleradores somente números inteiros, como o [Edge TPU](https://coral.ai/).

Neste tutorial, você vai treinar um modelo MNIST do zero, convertê-lo em um arquivo do TensorFlow Lite e aplicar a [quantização pós-treinamento](https://www.tensorflow.org/lite/performance/post_training_quantization). Por fim, você vai verificar a exatidão do modelo convertido e compará-la ao modelo float original.

Na verdade, você tem várias opções para o quanto quer quantizar o modelo. Neste tutorial, você vai realizar a "quantização de números inteiros", que converte todos os pesos e saídas de ativação em dados de números inteiros de 8 bits — enquanto outras estratégias podem deixar alguns dados em ponto flutuante.

Para saber mais sobre as diversas estratégias de quantização, leia sobre a [otimização de modelo do TensorFlow Lite](https://www.tensorflow.org/lite/performance/model_optimization).


## Configuração

Para quantizar ambos os tensores de entrada e saída, precisamos usar as APIs adicionadas no TensorFlow 2.3:

In [ ]:
import logging
logging.getLogger("tensorflow").setLevel(logging.DEBUG)

import tensorflow as tf
import numpy as np
print("TensorFlow version: ", tf.__version__)

## Gere um modelo do TensorFlow

Vamos criar um modelo simples para classificar números do [dataset MNIST](https://www.tensorflow.org/datasets/catalog/mnist).

Este treinamento não levará muito tempo, porque o modelo será treinado por apenas 5 épocas, com uma exatidão de aproximadamente 98%.

In [ ]:
# Load MNIST dataset
mnist = tf.keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalize the input image so that each pixel value is between 0 to 1.
train_images = train_images.astype(np.float32) / 255.0
test_images = test_images.astype(np.float32) / 255.0

# Define the model architecture
model = tf.keras.Sequential([
  tf.keras.layers.InputLayer(input_shape=(28, 28)),
  tf.keras.layers.Reshape(target_shape=(28, 28, 1)),
  tf.keras.layers.Conv2D(filters=12, kernel_size=(3, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(10)
])

# Train the digit classification model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(
                  from_logits=True),
              metrics=['accuracy'])
model.fit(
  train_images,
  train_labels,
  epochs=5,
  validation_data=(test_images, test_labels)
)

## Converta para um modelo do TensorFlow Lite

Agora você pode converter o modelo treinado para o formato do TensorFlow Lite usando o [Conversor](https://www.tensorflow.org/lite/models/convert) do TensorFlow Lite e aplicar níveis variados de quantização.

Esteja ciente de que algumas versões de quantização deixam alguns dados em formato float. Por isso, as seções a seguir mostram cada opção com níveis crescentes de quantização, até obter um modelo inteiramente de dados int8 ou uint8. (Observe que duplicamos um pouco do código em cada seção para você ver todas as etapas de quantização das opções.)

Primeiro, um modelo convertido sem nenhuma quantização:

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)

tflite_model = converter.convert()

Agora, é um modelo do TensorFlow Lite, mas ainda está usando valores float de 32 bits para todos os dados de parâmetros.

### Converta usando a quantização de intervalo dinâmico


Ative a flag `optimizations` padrão para quantizar todos os parâmetros fixos (como pesos):

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

tflite_model_quant = converter.convert()

Agora, o modelo está um pouco menor com pesos quantizados, mas os outros dados variáveis ainda estão em formato float.

### Converta usando a quantização com fallback de float

Para quantizar os dados variáveis (como a entrada/saída do modelo e os intermediários entre camadas), você precisa fornecer um [`RepresentativeDataset`](https://www.tensorflow.org/api_docs/python/tf/lite/RepresentativeDataset). Essa é uma função geradora que oferece um conjunto de dados de entrada grande o suficiente para representar valores típicos. Ela permite que o conversor estime um intervalo dinâmico para todos os dados variáveis. (O dataset não precisa ser único em relação ao dataset de treinamento ou avaliação.) Para aceitar várias entradas, cada ponto de dados representativo é uma lista, e os elementos na lista são alimentados ao modelo de acordo com os seus índices.


In [ ]:
def representative_data_gen():
  for input_value in tf.data.Dataset.from_tensor_slices(train_images).batch(1).take(100):
    # Model has only one input so each data point has one element.
    yield [input_value]

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen

tflite_model_quant = converter.convert()

Agora, todos os pesos e dados variáveis estão quantizados, e o modelo está significativamente menor em comparação com o original do TensorFlow Lite.

No entanto, para manter a compatibilidade com os aplicativos que usam tradicionalmente os tensores de entrada e saída do modelo float, o Conversor do TensorFlow Lite deixa esses tensores em float:

In [ ]:
interpreter = tf.lite.Interpreter(model_content=tflite_model_quant)
input_type = interpreter.get_input_details()[0]['dtype']
print('input: ', input_type)
output_type = interpreter.get_output_details()[0]['dtype']
print('output: ', output_type)

Geralmente, isso é bom para a compatibilidade, mas não será compatível com dispositivos que realizam operações baseadas somente em números inteiros, como o Edge TPU.

Além disso, o processo acima pode deixar uma operação no formato float se o TensorFlow Lite não incluir uma implementação quantizada para essa operação. Essa estratégia permite que a conversão seja concluída, para ter um modelo menor e mais eficiente, mas, novamente, não será compatível com hardware somente números inteiros. (Todas as operações nesse modelo MNIST têm uma implementação quantizada.)

Portanto, para garantir um modelo somente números inteiros de ponta a ponta, você precisa de mais alguns parâmetros...

### Converta usando a quantização somente números inteiros

Para quantizar os tensores de entrada e saída e fazer o conversor gerar um erro ao encontrar uma operação que não pode quantizar, converta o modelo novamente com alguns parâmetros adicionais:

In [ ]:
def representative_data_gen():
  for input_value in tf.data.Dataset.from_tensor_slices(train_images).batch(1).take(100):
    yield [input_value]

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen
# Ensure that if any ops can't be quantized, the converter throws an error
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# Set the input and output tensors to uint8 (APIs added in r2.3)
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8

tflite_model_quant = converter.convert()

A quantização interna permanece a mesma que acima, mas você pode ver que os tensores de entrada e saída estão agora no formato de números inteiros:


In [ ]:
interpreter = tf.lite.Interpreter(model_content=tflite_model_quant)
input_type = interpreter.get_input_details()[0]['dtype']
print('input: ', input_type)
output_type = interpreter.get_output_details()[0]['dtype']
print('output: ', output_type)

Agora, você tem um modelo quantizado de números inteiros que usa dados de números inteiros para os tensores de saída e entrada, para que seja compatível com hardware somente números inteiros, como o [Edge TPU](https://coral.ai).

### Salve os modelos como arquivos

Você precisará de um arquivo `.tflite` para implantar seu modelo em outros dispositivos. Portanto, vamos salvar os modelos convertidos em arquivos e carregá-los ao realizar as inferências abaixo.

In [ ]:
import pathlib

tflite_models_dir = pathlib.Path("/tmp/mnist_tflite_models/")
tflite_models_dir.mkdir(exist_ok=True, parents=True)

# Save the unquantized/float model:
tflite_model_file = tflite_models_dir/"mnist_model.tflite"
tflite_model_file.write_bytes(tflite_model)
# Save the quantized model:
tflite_model_quant_file = tflite_models_dir/"mnist_model_quant.tflite"
tflite_model_quant_file.write_bytes(tflite_model_quant)

## Execute os modelos do TensorFlow Lite

Agora, vamos realizar as inferências usando o [`Interpreter`](https://www.tensorflow.org/api_docs/python/tf/lite/Interpreter) do TensorFlow Lite para comparar as exatidões dos modelos.

Primeiro, precisamos de uma função que realize a inferência com um determinado modelo e imagens, para depois retornar as previsões:


In [ ]:
# Helper function to run inference on a TFLite model
def run_tflite_model(tflite_file, test_image_indices):
  global test_images

  # Initialize the interpreter
  interpreter = tf.lite.Interpreter(model_path=str(tflite_file))
  interpreter.allocate_tensors()

  input_details = interpreter.get_input_details()[0]
  output_details = interpreter.get_output_details()[0]

  predictions = np.zeros((len(test_image_indices),), dtype=int)
  for i, test_image_index in enumerate(test_image_indices):
    test_image = test_images[test_image_index]
    test_label = test_labels[test_image_index]

    # Check if the input type is quantized, then rescale input data to uint8
    if input_details['dtype'] == np.uint8:
      input_scale, input_zero_point = input_details["quantization"]
      test_image = test_image / input_scale + input_zero_point

    test_image = np.expand_dims(test_image, axis=0).astype(input_details["dtype"])
    interpreter.set_tensor(input_details["index"], test_image)
    interpreter.invoke()
    output = interpreter.get_tensor(output_details["index"])[0]

    predictions[i] = output.argmax()

  return predictions


### Teste os modelos em uma imagem


Agora, vamos comparar o desempenho dos modelos float e quantizado:

- `tflite_model_file` é o modelo original do TensorFlow Lite com os dados de ponto flutuante.
- `tflite_model_quant_file` é o último modelo que convertemos usando a quantização somente números inteiros (ele usa dados uint8 para a entrada e saída).

Vamos criar outra função para imprimir nossas previsões:

In [ ]:
import matplotlib.pylab as plt

# Change this to test a different image
test_image_index = 1

## Helper function to test the models on one image
def test_model(tflite_file, test_image_index, model_type):
  global test_labels

  predictions = run_tflite_model(tflite_file, [test_image_index])

  plt.imshow(test_images[test_image_index])
  template = model_type + " Model \n True:{true}, Predicted:{predict}"
  _ = plt.title(template.format(true= str(test_labels[test_image_index]), predict=str(predictions[0])))
  plt.grid(False)

Agora, teste o modelo float:

In [ ]:
test_model(tflite_model_file, test_image_index, model_type="Float")

E teste o modelo quantizado:

In [ ]:
test_model(tflite_model_quant_file, test_image_index, model_type="Quantized")

### Avalie os modelos em todas as imagens

Agora, vamos executar os dois modelos usando todas as imagens de teste que carregamos no início deste tutorial:

In [ ]:
# Helper function to evaluate a TFLite model on all images
def evaluate_model(tflite_file, model_type):
  global test_images
  global test_labels

  test_image_indices = range(test_images.shape[0])
  predictions = run_tflite_model(tflite_file, test_image_indices)

  accuracy = (np.sum(test_labels== predictions) * 100) / len(test_images)

  print('%s model accuracy is %.4f%% (Number of test samples=%d)' % (
      model_type, accuracy, len(test_images)))

Avalie o modelo float:

In [ ]:
evaluate_model(tflite_model_file, model_type="Float")

Avalie o modelo quantizado:

In [ ]:
evaluate_model(tflite_model_quant_file, model_type="Quantized")

Agora, você tem um modelo quantizado de números inteiros com quase nenhuma diferença na exatidão, em comparação com o modelo float.

Para saber mais sobre outras estratégias de quantização, leia sobre a [otimização de modelo do TensorFlow Lite](https://www.tensorflow.org/lite/performance/model_optimization).